<a href="https://colab.research.google.com/github/doorapr/dllab/blob/main/Masking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torchvision
import sys
!{sys.executable} -m pip install opencv-python matplotlib
!{sys.executable} -m pip install 'git+https://github.com/facebookresearch/segment-anything.git'

!wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth


In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import cv2
from numpy import loadtxt

In [ ]:
import sys
sys.path.append("..")
from segment_anything import sam_model_registry, SamPredictor

sam_checkpoint = "sam_vit_h_4b8939.pth"
model_type = "vit_h"

device = "cuda"

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)

predictor = SamPredictor(sam)

In [ ]:
def masking(image, masks):
  h,w,bpp = np.shape(image)
  newimage = image.copy()

  for py in range(0,h):
      for px in range(0,w):
#can change the below logic of rgb according to requirements. In this
#white background is changed to #e8e8e8  corresponding to 232,232,232
#intensity, red color of the image is retained.
          if(masks[0][py][px] == False):
              newimage[py][px][0]=225
              newimage[py][px][1]=225
              newimage[py][px][2]=225
  #newimage=cv2.cvtColor(newimage, cv2.COLOR_BGR2RGB)
  return newimage

In [ ]:
def mask_all(hd, i):
  #image = cv2.imread("Split_images/hd{:d}/wheel/cropped_original/0000{:02d}-img.jpg".format(hd, i))
  image = cv2.imread("Split_images/hd{:d}/wheel/cropped_original/0000{:02d}-img.jpg".format(hd, i))


  index1=loadtxt(f'index1_{i}.csv', delimiter=',')
  index2=loadtxt(f'index2_{i}.csv', delimiter=',')
  h,w,bpp = np.shape(image)
  newimage = image.copy()
  predictor.set_image(newimage)

  input_point = np.array([index1])
  input_label = np.array([1])


  masks, scores, logits = predictor.predict(
    point_coords=input_point,
    point_labels=input_label,
    multimask_output=True,
)


  input_point = np.array([index1,index2])
  input_label = np.array([1, 1])
  mask_input = logits[np.argmax(scores), :, :]

  masks, _, _ = predictor.predict(
    point_coords=input_point,
    point_labels=input_label,
    mask_input=mask_input[None, :, :],
    multimask_output=False,
)



  # Add function
  img = masking(newimage, masks)
  cv2.imwrite('Split_images/hd{:d}/wheel/segmented_original/0000{:02d}-img.jpg'.format(hd, i), img)

  image1 = cv2.imread("Split_images/hd{:d}/wheel/cropped_reconstructed/0000{:02d}-img.jpg".format(hd, i))
  img = masking(image1, masks)
  cv2.imwrite('Split_images/hd{:d}/wheel/segmented_reconstructed/0000{:02d}-img.jpg'.format(hd, i), img)


  image2 = cv2.imread("Split_images/hd{:d}/bulldozer/cropped_reconstructed/0000{:02d}-img.jpg".format(hd, i))
  img = masking(image2, masks)
  cv2.imwrite('Split_images/hd{:d}/bulldozer/segmented_reconstructed/0000{:02d}-img.jpg'.format(hd, i), img)

  image3 = cv2.imread("Split_images/hd{:d}/bulldozer/cropped_original/0000{:02d}-img.jpg".format(hd, i))
  img = masking(image3, masks)
  cv2.imwrite('Split_images/hd{:d}/bulldozer/segmented_original/0000{:02d}-img.jpg'.format(hd, i), img)
  return

In [ ]:
numbers = [1, 3, 4, 5, 6, 8, 9, 10, 13, 14, 15, 16, 17, 18]
hd = 8
hd_array = [1, 2, 4, 8, 16, 32, 64, 128]

for j in hd_array:
  for i in numbers:
    mask_all(j, i)

NameError: name 'mask_all' is not defined